In [ ]:
#sentiment of historcal figures from wikipiedia 
#Loading and Evaluating a Foundation Model

from transformers import AutoModelForCausalLM, AutoTokenizer
#from datasets import load_dataset xxxxxxxxxxxxxx

# import library
import stanza

# setup pipeline in en=english
stanza.download('en', processors='tokenize,pos')
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos')


# Load the foundation model
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)




# Load an appropriate dataset
dataset_name = "your/dataset"  # Replace with the actual dataset name
dataset = load_dataset(dataset_name)

# Example: Tokenize and evaluate
input_text = dataset['train'][0]['text']  # Use any text example from your dataset
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model(**inputs)
# Perform any necessary evaluation based on your task



In [ ]:
## Step 1: Load the dataset

We obtain the MNIST dataset using scikit-learn.

In [1]:

*`# Load MNIST using sklearn.datasets.fetch_openml`*

**`import** matplotlib.pyplot **as** plt
**import** numpy **as** np
**from** sklearn.datasets` 

**`import** fetch_openml

*# Load data from https://www.openml.org/d/43369*X, y **=** fetch_openml("mnist_784", version**=**1, return_X_y**=True**, parser**=**"auto")

*# Split into train and test*X_train, X_test **=** X[:60000], X[60000:]
y_train, y_test **=** y[:60000], y[60000:]
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

*# Convert to numpy arrays and scale for the model*X_train **=** np**.**array(X_train) **/** 255
X_test **=** np**.**array(X_test) **/** 255
y_train **=** np**.**array(y_train, dtype**=**np**.**int8)
y_test **=** np**.**array(y_test, dtype**=**np**.**int8)

*# Show the first 3 images*plt**.**figure(figsize**=**(20, 4))
**for** index, (image, label) **in** enumerate(zip(X_train[0:3], y_train[0:3])):
    plt**.**subplot(1, 3, index **+** 1)
    plt**.**imshow(np**.**reshape(image, (28, 28)), cmap**=**plt**.**cm**.**gray)
    plt**.**title("Label: %s\n" **%** label, fontsize**=**20)`

`Training set size: 60000
Test set size: 10000`

In [ ]:
#Performing Parameter-Efficient Fine-Tuning

from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments

# Create a PEFT config
lora_config = LoraConfig()

# Create a PEFT model
peft_model = get_peft_model(model, lora_config)

# Set up the training loop
training_args = TrainingArguments(
    output_dir="./peft_output",
    per_device_train_batch_size=2,  # Adjust as needed
    num_train_epochs=1,  # Adjust as needed
)

# Create a Trainer and fine-tune the model
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset['train'],  # Use your fine-tuning dataset
)

# Fine-tune the model
trainer.train()

# Save the trained PEFT model
peft_model.save_pretrained("gpt2-lora")



In [ ]:
#Performing Inference with a PEFT Model

from transformers import AutoPeftModelForCausalLM

# Load the trained PEFT model
peft_model = AutoPeftModelForCausalLM.from_pretrained("gpt2-lora")

# Perform inference
input_text = "Hello, my name is "  # Replace with your own input
inputs = tokenizer(input_text, return_tensors="pt")
outputs = peft_model.generate(input_ids=inputs["input_ids"], max_length=50)  # Adjust max_length as needed
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print or use the generated text as needed
print(generated_text)


In [1]:
pip install --upgrade transformers


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/85/f6/c5065913119c41ecad148c34e3a861f719e16b89a522287213698da911fc/transformers-4.37.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 979.8 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/28/03/7d3c7153113ec59cfb31e3b8ee773f5f420a0dd7d26d40442542b96675c3/huggingface_hub-0.20.3-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.19,>=0.14 from https://files.pythonhosted.org/packages/5f/4f/a4c12cc058a899c1caaa1e689c3df9a698e20e891d4005aa6ec2174a9339/tokenizers-0.15.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for safetensors>=0.4.1 from https://files.pythonhosted.org/packages/03/a0/67f2ed19dbc609ce1bf561a285a6cc8f804846a7f4d9c46b403be93022e4/safetensors-0.4.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obt